# S-MatCNGenPy

Este é um passo-a-passo da implementação em python do método **S-MatCNGenPy**, desenvolvido no trabalho \[1\]. O seu principal objetivo é garantir o suporte a referência ao esquema de dados em busca por palavras-chave em banco de dados. Observe que algumas consultas, como visto abaixo, não estão relacionadas apenas a valores do banco de dados, mas a própria estrutura do esquema.

```
    filmes do Will Smith
```
- **`filmes`** : relação Movie
- **`Will`, `Smith`** : instâncias da tabela Person(Name) 


#### Leituras Importantes

> [\[1\]](https://drive.google.com/file/d/1ZnljlKss9a8M_RDqseTYfZbQCjDhcJkk/view) MARTINS, Paulo Rodrigo O.; DA SILVA, Altigran Soares. *Uma Abordagem para Suporte a Referências ao Esquema em Consultas por Palavras-Chave em Bancos de Dados Relacionais*. Trabalho de Conclusão de Curso (Ciência da Computação), Universidade Federal do Amazonas, 2017. 

> [\[2\]]() DE OLIVEIRA, Pericles; DA SILVA, Altigran; DE MOURA, Edleno. *Match-Based Candidate Network Generation for Keyword Queries over Relational Databases*. In: Data Engineering (ICDE), 2018 IEEE 34st International Conference on. IEEE, 2016. Aceito pra Pubicação

> [\[3\]](https://dl.acm.org/citation.cfm?id=1989383) BERGAMASCHI, Sonia et al. *Keyword search over relational databases: a metadata approach*. In: Proceedings of the 2011 ACM SIGMOD International Conference on Management of data. ACM, 2011. p. 565-576.

In [1]:
import psycopg2
from psycopg2 import sql
from pprint import pprint as pp
from collections import defaultdict
import string
import itertools
import copy
from math import log1p
from queue import deque
import ast
import gc
from queue import deque

import nltk 
#nltk.download('wordnet')
#nltk.download('omw')
#nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

stw_set = set(stopwords.words('english')) - {'will'}

# Connect to an existing database
conn = psycopg2.connect("dbname=imdb user=postgres")

# Open a cursor to perform database operations
cur = conn.cursor()

/home/paulo/MatCNGenPy/venv/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


## Pré-processamento

Antes mesmo de receber os querysets, o sistema passa por um pré-processamento, que é responsavél pela criação de dois índices invertidos:

* **wordHash**: tabela que associa cada termo do banco de dados com o seu **IAF (Inverse Attribute Frequency)** e também referencia todas Tabelas, Colunas e CTIDs em que a palavra ocorre. Nota: o CTID é o endereço físico de uma linha em uma tabela, utilizado para encontrar rapidamente uma tupla.
```python
wordHash['term'] = ( IAF , { 'table': { 'column' : [ctid] } } )
```
* **attributeHash**: tabela que para cada atributo (documento), armazena a sua norma e o número de palavras distintas.
```python
attributeHash['table']['column'] = ( norm , num_distinct_words )
```

### Criação dos Índices Invertidos

O processo de criação é realizado em três etapas. Primeiramente, o procedimento ```createInvertedIndex()``` faz uma varredura no banco de dados e preenche parcialmente o ```wordHash```, faltando apenas calcular os IAFs para cada termo. Além disso, este procedimento também ele também armazena no ```attributeHash``` o total de palavras distintas para cada atributo.

Em seguida, os IAFs de cada termo são processados através do método ```processIAF(wordHash,attributeHash)```. Por último, as normas dos atributos (documentos) são calculadas no método ```processNormsOfAttributes(wordHash,attributeHash)```.

In [ ]:
def createInvertedIndex():
    #Output: wordHash (Term Index) with this structure below
    #map['word'] = [ 'table': ( {column} , ['ctid'] ) ]

    '''
    The Term Index is built in a preprocessing step that scans only
    once all the relations over which the queries will be issued.
    '''
    
    wordHash = {}
    attributeHash = {}
    
    # Get list of tablenames
    cur.execute("SELECT DISTINCT tablename FROM pg_tables WHERE schemaname!='pg_catalog' AND schemaname !='information_schema';")
    for table in cur.fetchall():
        table_name = table[0]
        print('INDEXING TABLE ',table_name)
        
        attributeHash[table_name] = {}
        
        #Get all tuples for this tablename
        cur.execute(
            sql.SQL("SELECT ctid, * FROM {};").format(sql.Identifier(table_name))
            #NOTE: sql.SQL is needed to specify this parameter as table name (can't be passed as execute second parameter)
        )

        for row in cur.fetchall():
            for column in range(1,len(row)):
                column_name = cur.description[column][0]   
                ctid = row[0]

                for word in [word.strip(string.punctuation) for word in str(row[column]).lower().split()]:
                    
                    #Ignoring STOPWORDS
                    if word in stw_set:
                        continue

                    #If word entry doesn't exists, it will be inicialized (setdefault method),
                    #Append the location for this word
                    wordHash.setdefault(word, {})                    
                    wordHash[word].setdefault( table_name , {} )
                    wordHash[word][table_name].setdefault( column_name , [] ).append(ctid)
                    
                    attributeHash[table_name].setdefault(column_name,(0,set()))
                    attributeHash[table_name][column_name][1].add(word)
        
        #Count words
        
        for (column_name,(norm,wordSet)) in attributeHash[table_name].items():
            num_distinct_words = len(wordSet)
            wordSet.clear()
            attributeHash[table_name][column_name] = (norm,num_distinct_words)
        

    print ('INVERTED INDEX CREATED')
    return (wordHash,attributeHash)

(wordHash,attributeHash) = createInvertedIndex()

In [ ]:
pp(wordHash['denzel'])

In [ ]:
pp(attributeHash['movie'])

In [ ]:
def processIAF(wordHash,attributeHash):
    
    total_attributes = sum([len(attribute) for attribute in attributeHash.values()])
    
    for (term, values) in wordHash.items():
        
        attributes_with_this_term = sum([len(attribute) for attribute in wordHash[term].values()])
        
        IAF = log1p(total_attributes/attributes_with_this_term)
                
        wordHash[term] = (IAF,values)
    print('IAF PROCESSED')

processIAF(wordHash,attributeHash)

In [ ]:
pp(wordHash['denzel'])

In [ ]:
def processNormsOfAttributes(wordHash,attributeHash):
  
    # Get list of tablenames
    cur.execute("SELECT DISTINCT tablename FROM pg_tables WHERE schemaname!='pg_catalog' AND schemaname !='information_schema';")
    for table in cur.fetchall():
        table_name = table[0]
        print('PROCESSING TABLE ',table_name)
        
        #Get all tuples for this tablename
        cur.execute(
            sql.SQL("SELECT ctid, * FROM {};").format(sql.Identifier(table_name))
            #NOTE: sql.SQL is needed to specify this parameter as table name (can't be passed as execute second parameter)
        )

        for row in cur.fetchall():
            for column in range(1,len(row)):
                column_name = cur.description[column][0]   
                ctid = row[0]

                for word in [word.strip(string.punctuation) for word in str(row[column]).lower().split()]:
                    
                    #Ignoring STOPWORDS
                    if word in stw_set:
                        continue
                    
                    (prevNorm,num_distinct_words)=attributeHash[table_name][column_name]
                    
                    IAF = wordHash[word][0]
                    
                    Norm = prevNorm + IAF
                    
                    attributeHash[table_name][column_name]=(Norm,num_distinct_words)
                    

    print ('NORMS OF ATTRIBUTES PROCESSED')

processNormsOfAttributes(wordHash,attributeHash)

In [ ]:
pp(attributeHash['movie'])

## Main

O processamento das consultas é realizado em 

In [ ]:
def getQuerySets():
    QuerySet = []
    with open('querysets/queryset_imdb_martins.txt') as f:
        for line in f.readlines():
            
            #The line bellow Remove words not in OLIVEIRA experiments
            Q = [word.strip(string.punctuation) for word in line.split() if word not in ['title','dr.',"here's",'char','name'] and word not in stw_set]  
            
            #Q = [word.strip(string.punctuation) for word in line.split() if word not in stw_set]  
            
            QuerySet.append(Q)
    return QuerySet
        
QuerySet = getQuerySets()
QuerySet

### Recuperação de Tuple-sets
Esta etapa consiste em recuperar conjuntos de tuplas que contém cada palavra-chave, chamados de tuple-sets. O algoritmo `TSFind`, que realiza esse processo, pode ser é divido em três partes: 
* **Recuperação de tuplas:** Essa parte consiste em encontrar os conjuntos de tuplas que contém cada uma das palavras do Queryset. Essas informações já foram pré-processadas no índice invertido `wordHash`.
* **Interseção de tuplas:** Esta parte acontece no algoritmo `TSInter` e é responsável por encontrar tuplas que contém mais de uma das palavras-chave. Além disso, esta etapa irá garantir que os tuple-sets `TABLE{word}` contenham apenas a palavra `word` e nenhuma outra palavra do queryset. Esta propriedade é necessária para encontrar a cobertura mínima (etapa de criação de query matches). 
* **Criação de tuple-sets:** Esta parte irá condensar os resultados. Em vez de listar todas as tuplas que contenham as palavras-chave, precisamos apenas saber quais colunas possuem cada uma das palavras. Por isso, os tuple-sets terão a estrutura (o primeiro atributo refere-se a *value* ou *schema*):
```python
TupleSet = ('v','table','column', frozenset({words}))
```

In [ ]:
def TSFind(Q):
    #Input:  A keyword query Q=[k1, k2, . . . , km]
    #Output: Set of non-free and non-empty tuple-sets Rq

    '''
    The tuple-set Rki contains the tuples of Ri that contain all
    terms of K and no other keywords from Q
    '''
    
    #Part 1: Find sets of tuples containing each keyword
    global P
    P = {}
    for keyword in Q:
        tupleset = set()
        
        if keyword not in wordHash:
            continue
        
        for (table,attributes) in wordHash.get(keyword)[1].items():
            for (attribute,ctids) in attributes.items():
                for ctid in ctids:
                    tupleset.add( (table,attribute,ctid) )
        P[frozenset([keyword])] = tupleset
    
    #Part 2: Find sets of tuples containing larger termsets
    P = TSInter(P)
    
    #Part 3:Build tuple-sets
    Rq = set()
    for keyword , tuples in P.items():
        for (table,attribute,ctid) in tuples:
            Rq.add( ('v',table,attribute,keyword) )
    print ('TUPLE SETS CREATED')
    return Rq


def TSInter(P):
    #Input: A Set of non-empty tuple-sets for each keyword alone P 
    #Output: The Set P, but now including larger termsets (process Intersections)

    '''
    Termset is any non-empty subset K of the terms of a query Q        
    '''
    
    Pprev = {}
    Pprev=copy.deepcopy(P)
    Pcurr = {}

    combinations = [x for x in itertools.combinations(Pprev.keys(),2)]
    for ( Ki , Kj ) in combinations:
        Tki = Pprev[Ki]
        Tkj = Pprev[Kj]
        
        X = Ki | Kj
        Tx = Tki & Tkj        
        
        if len(Tx) > 0:            
            Pcurr[X]  = Tx            
            Pprev[Ki] = Tki - Tx         
            Pprev[Kj] = Tkj - Tx
            
    if Pcurr != {}:
        Pcurr = copy.deepcopy(TSInter(Pcurr))
        
    #Pprev = Pprev U Pcurr
    Pprev.update(Pcurr)     
    return Pprev   

Q = ['actor', 'james', 'bond']
Rq = TSFind(Q)
pp(Rq)

### Criação Schema-sets

Esta etapa consiste na criação dos Schema-sets, que é uma estrutura análoga aos tuple-sets vistos na etapa anterior. Aqui, o processo também é divido em três partes: 
* **Mapeamento de Elementos do Esquema (*Schema Matching*):** Essa parte consiste em analisar a similaridade entre as palavras do querysets e elementos do esquema (nomes de relações e atributos).
* **Análise de Termos Adjacentes:** Esta parte verifica as relações entre as palavras chave, muitas vezes uma palavras-chave relacioada a elemento do esquema delimita o domínio das palavras-chave adjacentes. Ex: Actor James Bond delimita a palavra James para nome de Pessoa, em vez de nome de Filme.
* **Criação de Schema-sets:** Esta parte irá formatar os resultados para ficarem semelhantes à estrutura de tuple-sets, seguindo a estrutura a seguir (o primeiro atributo refere-se a *value* ou *schema*):
```python
SchemaSet = ('s','table','column', frozenset({words}))
```

#### Similaridades para o Schema-Matching

Para o mapeamento de palavras para elementos do esquema, foram utilizadas métricas de similaridade de escrita e semântica.
O Coeficiente de Jaccard é uma métrica que avalia a interseção entre duas palavras, sendo ideal para similaridades de escrita, como abreviações ou erros de digitação. 

Por outro lado, as métricas semânticas utilizam o dicionário léxico WordNet para encontrar similaridades de sentido. O pacote de ferramentas NLTK disponibiliza uma série de métricas semânticas [aqui](http://www.nltk.org/howto/wordnet.html "WordNet Interface"). Entre elas, as principais são a Path Similarity e a Wu-Palmer Similarity. A primeira métrica procura encontrar a menor distância entre duas palavras, no grafo de relações do WordNet, enquanto a segunda analisa o ancestral comum mais próximo entre duas palavras.

In [ ]:
def wordNetSimilarity(wordA,wordB):
    
    A = set(wn.synsets(wordA))
    B = set(wn.synsets(wordB))
    
    wupSimilarities = [0]
    pathSimilarities = [0]
    for (sense1,sense2) in itertools.product(A,B):        
        wupSimilarities.append(wn.wup_similarity(sense1,sense2) or 0)
        pathSimilarities.append(wn.path_similarity(sense1,sense2) or 0)
    return max(max(wupSimilarities),max(pathSimilarities))

def jaccard_similarity(wordA,wordB):
    
    A = set(wordA)
    B = set(wordB)
    
    return len(A & B ) / len(A | B)
    
def wordSimilarity(wordA,wordB):
    return max( (jaccard_similarity(wordA,wordB),wordNetSimilarity(wordA,wordB)) )

In [ ]:
set(wn.synsets('come'))

#### Algoritmo para Criação dos Schema-Sets

In [ ]:
def SchSFind(Q,threshold):
    S = []
    
    for (position,keyword) in enumerate(Q):
        for (table,values) in attributeHash.items():
            
            sim = wordSimilarity(keyword,table)
            if sim >= threshold:
                S.append( (table,'*',{keyword},position,sim) )
            
            for attribute in values.keys():
                
                if(attribute=='id'):
                    continue
                
                sim = wordSimilarity(keyword,attribute)
                
                if sim >= threshold:
                    S.append( (table,attribute,{keyword},position,sim) )
    #S = SchSInter(S)

    print ('SCHEMA SETS CREATED')
    Sq = {('s',table,attribute,frozenset(keywords)) for (table,attribute,keywords,position,sim) in S}
        
    return Sq

'''
Em vez de interseções, deve ser feita uma análise dos adjacentes..

def SchSInter(S):
    
    Scurr= S.copy()
    
    somethingChanged = False

    combinations = [x for x in itertools.combinations(Scurr,2)]
    
    for ( A , B ) in combinations:    
    
        (tableA,attributeA,wordsA,positionA,simA) = A
        (tableB,attributeB,wordsB,positionB,simB) = B
        
        if A not in Scurr or B not in Scurr:
            continue
        
        if tableA == tableB and abs(positionA-positionB)<=1:
            print('A:\n',A)
            print('B:\n',B)
            
            AB = (tableA, '*' , wordsA | wordsB, max((positionA,positionB)) , max((simA,simB)) )
            
            Scurr.remove(A)
            Scurr.remove(B)
            Scurr.append(AB)
            
            somethingChanged = True 
   
    if somethingChanged:
        return SchSInter(Scurr)
    
    return Scurr
'''
''

In [ ]:
Q = QuerySet[5] = ['actor', 'james', 'bond']
SimilarityCoeficient = 0.799999999999
Sq = SchSFind(Q,SimilarityCoeficient)
Sq

### Criação de Query Matches

As etapas anteriores, de criação de schema-sets e tuple-sets, foram responsáveis por identificar quais relações possuem alguma informação sobre as palavras-chave. Nesta etapa de criação de full matches, o objetivo é combinar esses tuple-sets e schema-sets para se obter uma resposta completa, mínima e relevante para o usuário. 

O algoritmo `QMGen` é responsável por encontrar combinações de tuple-sets/schema-sets que compõem uma cobertura mínima (`MinimalCover`) sobre o queryset.
- **Total**: Cada palavra-chave deve estar presente em ao menos uma das tuplas da query-match.
- **Mínima**: Não é possível remover nenhum tuple-set/schema-set da query-match e manter a cobertura total sobre o queryset.

In [ ]:
def MinimalCover(MC, Q):
    #Input:  A subset MC (Match Candidate) to be checked as total and minimal cover
    #Output: If the match candidate is a TOTAL and MINIMAL cover

    Subset = [termset for category,table,attribute,termset in MC]
    u = set().union(*Subset)    
    
    isTotal = (u == set(Q))
    for element in Subset:
        
        new_u = list(Subset)
        new_u.remove(element)
        
        new_u = set().union(*new_u)
        
        if new_u == set(Q):
            return False
    
    return isTotal

In [ ]:
def QMGen(Q,Rq):
    #Input:  A keyword query Q, The set of non-empty non-free tuple-sets Rq
    #Output: The set Mq of query matches for Q
    
    '''
    Query match is a set of tuple-sets that, if properly joined,
    can produce networks of tuples that fulfill the query. They
    can be thought as the leaves of a Candidate Network.
    
    '''
    
    Mq = []
    for i in range(1,len(Q)+1):
        for subset in itertools.combinations(Rq,i):
            if(MinimalCover(subset,Q)):
                Mq.append(set(subset))
    return Mq

In [ ]:
Q =['actor', 'draco', 'harry','potter']

Rq = TSFind(Q)
SimilarityCoeficient = 0.799999999999
Sq = SchSFind(Q,SimilarityCoeficient)

Mq = QMGen(Q,Sq|Rq)
for match in Mq:
    pp(match)
    print('\n\n')

### Criação de Candidate Networks

Na etapa anterior, obteve-se as full matches, que compreendem todas as informações necessárias para o usuário. O próximo passo é encontrar maneiras de conectar estas informações para formar uma resposta para o usuário. Estas conexões, chamadas de candidate networks, são derivadas das restrições de integridade referencial do banco de dados, também conhecidas como chaves estrangeiras.

A criação de candidate networks utiliza dois grafos:
- **Schema Graph**: vértice que representa o banco de dados e é utilizado como base para o match graph. Ele contém como vértices os free tuple-sets associados a cada relação do banco de dados e como arestas as restrições de integridade referencial.

    O Schema Graph foi implementado como um dicionário, no qual cada vértice aponta para um outro vértice. Além disso, também é armazenada informações sobre as arestas, como direção e quais atributos entre as tabelas tem a relação de restrição referencial. A estrutura do Schema Graph pode ser observada a seguir:
   
```python
    G['table'] = { 'foreign_table' : (direction, column, foreign_column) }
```

In [ ]:
def getSchemaGraph():
    #Output: A Schema Graph G  with the structure below:
    # G['node'] = edges
    # G['table'] = { 'foreign_table' : (direction, column, foreign_column) }
    
    
    G = {} 
    cur.execute("SELECT tablename FROM pg_tables WHERE schemaname!='pg_catalog' AND schemaname !='information_schema';")
    for table in cur.fetchall():
        G.setdefault(table[0],{})
    
    sql = "SELECT DISTINCT                 tc.table_name, kcu.column_name,                 ccu.table_name AS foreign_table_name, ccu.column_name AS foreign_column_name             FROM information_schema.table_constraints AS tc              JOIN information_schema.key_column_usage AS kcu                 ON tc.constraint_name = kcu.constraint_name             JOIN information_schema.constraint_column_usage AS ccu                 ON ccu.constraint_name = tc.constraint_name             WHERE constraint_type = 'FOREIGN KEY'"
    cur.execute(sql)
    relations = cur.fetchall()
    
    for (table,column,foreign_table,foreign_column) in relations:
        G[table][foreign_table] = (1,column, foreign_column)
        G[foreign_table][table] = (-1,foreign_column,column)
    print ('SCHEMA CREATED')
    return G
G = getSchemaGraph()
G


- **Match Graph**: grafo gerado a partir de uma query match e o schema graph. No entanto, no match graph tuple-sets/schema-sets também são modelados como vértices. Para criá-lo, adiciona-se ao schema graph os tuple-sets/schema-sets presentes na query match. Um tuple-set de uma tabela x terá os mesmos relacionamentos (arestas) que o vértice x.

```python
    Gts['table'] = { 'foreign_table' : (direction, column, foreign_column) }

    Gts[('s','table','column', frozenset({words}))] = { 'foreign_table' : (direction, column, foreign_column) }
```

In [ ]:
def MatchGraph(Rq, G, M):
    #Input:  The set of non-empty non-free tuple-sets Rq,
    #        The Schema Graph G,
    #        A Query Match M
    #Output: A Schema Graph Gts  with the structure below:
    # G['node'] = edges
    # G['table'] = { 'foreign_table' : (direction, column, foreign_column) }

    '''
    A Match Subgraph Gts[M] is a subgraph of G that contains:
        The set of free tuple-sets of G
        The query match M
    '''
    
    Gts = copy.deepcopy(G)
    
    tables = set()
    #Insert non-free nodes
    for (category,table ,attribute, keywords) in M:
        Gts[(category,table,attribute,keywords)]=copy.deepcopy(Gts[table])
        for foreign_table , (direction,column,foreign_column) in Gts[(category,table,attribute,keywords)].items():
            Gts[foreign_table][(category,table,attribute,keywords)] = (direction*(-1),foreign_column,column)

    return Gts 

In [ ]:
M = {('s', 'person', '*', frozenset({'actor'})),
 ('v', 'char', 'name', frozenset({'draco'})),
 ('v', 'movie', 'title', frozenset({'potter', 'harry'}))}

Gts = MatchGraph(Rq|Sq, G, M)
pp(Gts)

#### Algoritmo para Criação de Candidate Networks

Para criar uma candidate network, o algoritmo `SingleCN` procura um caminho mínimo no match graph que visite todas os non-free tuple-sets/schema-sets da query match. 

Este caminho deve ser:
- **Mínimo:** garantido através do algoritmo de caminho mínima baseado em busca por largura (BFS).
- **Total:** a função `containsMatch` garante que todos os tuple-sets/schema-sets da query match sejam visitados.
- **Seguro (*Sound*):** uma joining networks of tuple-sets é considerado sound se ela não contém uma subárvore na forma $R^K - S^L - R^M $, na qual $R$ e $S$ são relações e o schema graph tem uma aresta $R \rightarrow S$.

In [ ]:
def containsMatch(Ji,M):
    for relation in M:
        if relation not in Ji:
            return False
    return True

def isJNTSound(Gts,Ji):
    if len(Ji)<3:
        return True
    
    for i in range(len(Ji)-2):
        
        if type(Ji[i]) is str:
            tableA = Ji[i]
        else:
            tableA = Ji[i][0]
            
        if type(Ji[i+2]) is str:
            tableB = Ji[i+2]
        else:
            tableB = Ji[i+2][0]          
            
        if tableA==tableB:
            edge_info = Gts[Ji[i]][Ji[i+1]]
            if(edge_info[0] == -1):
                return False
    return True

In [ ]:
def SingleCN(FM,Gts,Tmax):    
    '''
    print('================================================================================\nSINGLE CN')
    print('Tmax ',Tmax)
    print('FM')
    pp(FM)
    
    print('\n\nGts')
    pp(Gts)
    print('\n\n')
    '''
    F = deque()

    first_element = list(FM)[0]
    J = [first_element]
    
    if len(FM)==1:
        return J
    
    F.append(J)
    
    while F:
        J = F.popleft()           
        u = J[-1]
        '''
        print('--------------------------------------------\nParctial CN')
        print('J ',J,'\n')
        
        print('\nAdjacents:')
        pp(Gts[u].items())
        '''
        for (adjacent,edge_info) in Gts[u].items():
            if (type(adjacent) is str) or (adjacent not in J):
                Ji = J + [adjacent]
                if (Ji not in F) and (len(Ji)<Tmax) and (isJNTSound(Gts,Ji)):
                    if(containsMatch(Ji,FM)):
                        '''
                        print('--------------------------------------------\nGenerated CN')
                        print('J ',Ji,'\n')
                        '''
                        return Ji
                    else:
                        F.append(Ji)

In [ ]:
SingleCN(M,Gts,10)

In [ ]:
def MatchCN(G,Sq,Rq,Mq):    
    Cns = []                        
    for M in Mq:
        Gts = MatchGraph(Rq|Sq, G, M)
        Cn = SingleCN(M,Gts,10)
        if(Cn is not None):
            Cns.append( (Cn,Gts,M) )
    return Cns


Cns = MatchCN(G,Rq,Sq,Mq)
for Cn in Cns:
    pp(Cn)
    print('--------------------------------------------------------------\n')

### Ranking de Candidate Networks

Como existem diferentes maneiras de se conectar as informações associadas as palavras-chave, várias candidate networks serão geradas. Entretanto, na maioria das vezes, apenas uma delas contém uma resposta relevante para o usuário. Por este motivo, esta esta etapa irá avaliar e ranquear as candidate networks por relevância.

In [ ]:
def CNRank(Cns,mi,smi):
    Ranking = []
    for (Cn,Gts,M) in Cns:
        cosprod = 1
        valuecont = 0

        schemaprod = 1
        schemacont = 0
        
        for relation in Cn:
            if(type(relation) is str):
                continue
            
            (category,table,attribute,predicates) = relation
            
            if (category == 'v'):
                
                valuecont+=1
                
                if predicates == frozenset(['']):
                    continue

                (norm_attribute,distinct_terms) = attributeHash[table][attribute]

                wsum = 0

                for term in predicates:

                    IAF = wordHash[term][0]

                    ctids = wordHash[term][1][table][attribute]
                    fkj = len(ctids)

                    if fkj>0:

                        TF = log1p(fkj) / log1p(distinct_terms)

                        wsum = wsum + TF*IAF
                
                cos = wsum/norm_attribute
                cosprod *= cos
            elif (category == 's'):
                
                schemacont+=1
                
                if(attribute == '*'):
                    schemaTerm = table
                else:
                    schemaTerm = attribute
                
                schemasum = 0
                
                for term in predicates:
                    schemasum+=wordSimilarity(term, schemaTerm)
                
                schemaprod *= schemasum
                
            
        
        valuescore = schemascore = 0
        
        score = 1/len(Cn)
        
        if valuecont>0:
            valuescore = mi * cosprod 
            score*=valuescore
        
        if schemacont>0:
            schemascore = smi * schemaprod
            score*=schemascore
            
        Ranking.append((Cn,Gts,M,score , ( valuescore , schemascore , len(Cn) )  ))
    return sorted(Ranking,key=lambda x: x[3],reverse=True)

In [ ]:
mi = 464576.1086662768
smi = 1

SortedCn = CNRank(Cns,mi,smi)
SortedCn

for Cn in SortedCn:
    print('Value Score: ',"%.8f" % Cn[4][0],'\nSchema Score:',"%.8f" % Cn[4][1], '\n|Cn|: ',Cn[4][2],'\nTotal Score: ',"%.8f" % Cn[3])
    pp(Cn[0])
    print('----------------------------------------------------------------------\n')

### Mais abaixo tem a execução para outras CNS (querysets)

# Execução

In [ ]:
def preProcessing():
    global wordHash
    global attributeHash
    (wordHash,attributeHash) = createInvertedIndex()
    processIAF(wordHash,attributeHash)
    processNormsOfAttributes(wordHash,attributeHash)
    print('PRE-PROCESSING STAGE FINISHED')

In [ ]:
def main(mi,smi):   
    QuerySets = getQuerySets()
    maxscores = (list(),list())
    for (i,Q) in enumerate(QuerySets):
       
        print('QUERY-SET ',Q,'\n')
        
        print('FINDING TUPLE-SETS')
        Rq = TSFind(Q)
        print(len(Rq),'TUPLE-SETS CREATED\n')
        
        print('FINDING SCHEMA-SETS')
        SimilarityThreshold = 0.799999999999
        Sq = SchSFind(Q,SimilarityThreshold)
        print(len(Sq),' SCHEMA-SETS CREATED\n')
        
        print('GENERATING QUERY MATCHES')
        Mq = QMGen(Q,Sq|Rq)
        print (len(Mq),'QUERY MATCHES CREATED\n')
        '''
        for M in Mq[:20]:
            pp(M)
            print('\n\n')
        '''
        print('GENERATING CANDIDATE NETWORKS')
        G = getSchemaGraph()
        
        Cns = MatchCN(G,Rq,Sq,Mq)
        
        print (len(Cns),'CANDIDATE NETWORKS CREATED\n')
        
        '''
        for Cn in Cns[:20]:
            pp(Cn[0])
            print('\n\n')
            #pp(Cn[1])
            #print('\n\n\n==================================================================================\n')
        '''
        print('RANKING CANDIDATE NETWORKS')
        RankedCns = CNRank(Cns,mi,smi)
        for (j,Cn) in enumerate(RankedCns):
            print(j+1,'ª CN')
            print('Value Score: ',"%.8f" % Cn[4][0],'\nSchema Score:',"%.8f" % Cn[4][1], '\n|Cn|: ',Cn[4][2],'\nTotal Score: ',"%.8f" % Cn[3])
            pp(Cn[0])
            print('----------------------------------------------------------------------\n')
        
            maxscores[0].append(Cn[4][0])
            maxscores[1].append(Cn[4][1])
        gc.collect()
        
        print('==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================')
    return maxscores

In [ ]:
mi = 0.90/1.9372498568291752e-06
mi

In [ ]:
mi = 464576.1086662768
smi = 1
maxscores = main(mi,smi)
maxscores

### Observando os maxscores que eu atribui valores a constante mi para normalizar o resultado

In [ ]:
max(maxscores[0]), max(maxscores[1])

In [ ]:
def detailedInfo(Q):   
    print('QUERY-SET ',Q,'\n')

    print('FINDING TUPLE-SETS')
    Rq = TSFind(Q)
    print(len(Rq),'TUPLE-SETS CREATED\n')

    print('FINDING SCHEMA-SETS')
    SimilarityThreshold = 0.799999999999
    Sq = SchSFind(Q,SimilarityThreshold)
    print(len(Sq),' SCHEMA-SETS CREATED\n')

    print('GENERATING QUERY MATCHES')
    Mq = QMGen(Q,Sq|Rq)
    print (len(Mq),'QUERY MATCHES CREATED\n')
    
    for M in Mq[:20]:
        pp(M)
        print('\n\n')
    
    print('GENERATING CANDIDATE NETWORKS')
    G = getSchemaGraph()

    Cns = MatchCN(G,Rq,Sq,Mq)

    print (len(Cns),'CANDIDATE NETWORKS CREATED\n')

    '''
    for Cn in Cns[:20]:
        pp(Cn[0])
        print('\n\n')
        #pp(Cn[1])
        #print('\n\n\n==================================================================================\n')
    '''
    print('RANKING CANDIDATE NETWORKS')
    mi = 2.7E+12
    smi = 1
    RankedCns = CNRank(Cns,mi,smi)
    for (j,Cn) in enumerate(RankedCns):
        print(j+1,'ª CN')
        print('Value Score: ',"%.8f" % Cn[4][0],'\nSchema Score:',"%.8f" % Cn[4][1], '\n|Cn|: ',Cn[4][2],'\nTotal Score: ',"%.8f" % Cn[3])
        pp(Cn[0])
        print('----------------------------------------------------------------------\n')

    gc.collect()

    print('==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================\
==========================================================================')

In [ ]:
detailedInfo(['ellen','page'])

## Todos os mapeamentos de elementos para esquema

In [ ]:
for Q in getQuerySets():
    Sq = SchSFind(Q,0.8)
    print(Q)
    pp(Sq)
    print('\n\n')
    